In [1]:
import os
import logging

# Octave engine
from oct2py import Oct2Py, get_log
engine1 = Oct2Py(logger=get_log())
engine1.logger = get_log("new_log")
engine1.logger.setLevel(logging.WARNING)

# Matlab engine
import matlab.engine
engine2 = matlab.engine.start_matlab()

# Test
res1 = engine1.eval("test_matlab_func_args(3, 4)")
res2 = engine1.test_matlab_func_args(3, 4)

res3 = engine2.eval("test_matlab_func_args(3, 4)")
res4 = engine2.test_matlab_func_args(3, 4)

print(res1)
print(res2)
print(res3)
print(res4)

-1.0
-1.0
-1.0
-1


In [7]:
engine2.eval("addpath('/Users/arno/Python/eegprep/src/eegprep/eeglab')")
engine2.eval("eeglab nogui;", nargout=0)
engine2.eval("EEG = pop_loadset('/Users/arno/Python/eegprep/data/FlankerTest.set')", nargout=0)
engine2.eval("EEG = pop_select(EEG, 'rmtime', [75.35900000000001 150.68800000000002])", nargout=0)
res = engine2.eval('EEG.pnts;')
res


eeglab: options file is ~/eeg_options.m
Retrieving plugin versions from server...
Retrieving download statistics...
Issue with retrieving statistics for extensions
EEGLAB: adding "Biosig" v3.8.4 to the path
EEGLAB: adding "EEG-BIDS" v? (see >> help eegplugin_eegbids)
EEGLAB: adding "ICLabel" v? (see >> help eegplugin_iclabel)
EEGLAB: adding "clean_rawdata" v? (see >> help eegplugin_clean_rawdata)
EEGLAB: adding "dipfit" v? (see >> help eegplugin_dipfit)
EEGLAB: adding "firfilt" v? (see >> help eegplugin_firfilt)
pop_loadset(): loading file /Users/arno/Python/eegprep/data/FlankerTest.set ...

EEG = 

  struct with fields:

             setname: ''
            filename: 'FlankerTest.set'
            filepath: '/Users/arno/Python/eegprep/data'
             subject: ''
               group: ''
           condition: ''
             session: []
            comments: ''
              nbchan: 64
              trials: 1
                pnts: 75350
               srate: 100
                xmin:

67816.0

In [ ]:
# passing arrays to matlab functions

import numpy as np

res1 = engine1.eval("test_matlab_func_args2([3 3], [4 4])")
res2 = engine1.test_matlab_func_args2(np.array([3,3]), np.array([4,4])) # note that [3,4] get pass as a cell array
res3 = engine1.test_matlab_func_args2([[3,3]], [[4,4]])

res4 = engine2.eval("test_matlab_func_args2([3 3], [4 4])")
res5 = engine2.test_matlab_func_args2(np.array([3,3]), np.array([4,4]))
#res6 = engine2.test_matlab_func_args2([[3,3]], [[4,4]]) # this does not work

print(res1)
print(res2)
print(res3)
print(res4)
print(res5)
#print(res6)


7.0
7.0
7.0
7.0
7


In [ ]:
# passing structs to matlab functions and returning numpy arrays
res1 = engine1.eval("test_matlab_func_args3(struct('value', [3 3]))")
print(res1)
print(type(res1))

res2 = engine2.eval("test_matlab_func_args3(struct('value', [3 3]))") # this one can print the type of the result
print(res2)
# show the type of the result
print(type(res2))
res3 = np.array(res2)
print(res3)
print(type(res3))



[[3. 3.]]
<class 'numpy.ndarray'>


In [ ]:
# check passing structs as dictionaries to matlab functions
res1 = engine1.test_matlab_func_args3({'value': [3, 3]})
print(res1)

res2 = engine2.test_matlab_func_args3({'value': [3, 3]}) # this one can print the type of the result
print(res2)


[[3. 3.]]
[3, 3]


in1 is a struct


In [ ]:
import numpy as np
from oct2py import Oct2Py

oc = Oct2Py()
dt = np.dtype([('y', float), ('z', float)])
arr = np.array([(1.0, 3.0), (2.0, 4.0)], dtype=dt)  # 2x1 struct array
oc.push('x', arr)                                   # x is a struct array in Octave

In [ ]:
from scipy.io import savemat
savemat('x.mat', {'x': arr})

In [ ]:
from oct2py import Oct2Py
oc = Oct2Py()
oc.eval('x = struct("y", {1, 2}, "z", {3, 4});')
X = oc.pull('x')            # X is oct2py.StructArray (np.recarray subclass)
X.y                          # -> Cell([[1.0, 2.0]])
X[0, 1].z                    # -> 4.0 (numpy scalar)

In [48]:
# check passing structs as dictionaries to matlab functions
#res1 = engine1.test_matlab_func_args4([{'value': [3, 3]}, {'value': [4, 4]}])
#print(res1)

res2 = engine2.test_matlab_func_args4([[{'value': [3, 3]}, {'value': [4, 4]}]])
print(res2)

in1 is a cell
Index exceeds the number of array elements. Index must not exceed 1.

Error in test_matlab_func_args4 (line 3)
    out1 = in1(2).value;



MatlabExecutionError: 
  File /Users/arno/Python/eegprep/notebooks/test_matlab_func_args4.m, line 3, in test_matlab_func_args4
Index exceeds the number of array elements. Index must not exceed 1.


In [45]:
import array
res = array.array([{'value': [3, 3]}, {'value': [4, 4]}])
res[0]

TypeError: array() argument 1 must be a unicode character, not list

In [50]:
# Python "structure array"
people = [
    {"name": "Alice", "age": 30, "height": 1.67},
    {"name": "Bob",   "age": 25, "height": 1.80}
]

# Pass to MATLAB
engine2.workspace['people'] = people

# Inspect and use in MATLAB
engine2.eval("whos('people')", nargout=0)
engine2.eval("disp(people(1).name)", nargout=0)

# Use from MATLAB code (example: compute mean height)
mean_h = engine2.eval("mean([people.height])")
print(mean_h)  # 1.735

  Name        Size            Bytes  Class    Attributes

  people      1x2              1264  cell               



Dot indexing is not supported for variables of this type.



MatlabExecutionError: Dot indexing is not supported for variables of this type.


In [53]:

# Option B: construct a struct array directly via MATLAB's struct
people = engine1.struct(
    'name', ['Alice','Bob'],        # Python list -> MATLAB cell array of char vectors
    'age',  [30.0, 25.0]            # Python list -> MATLAB cell array of doubles
)
engine1.workspace['people'] = people

engine1.eval("disp(people(1).name)", nargout=0)

TypeError: 'workspace' object does not support item assignment